In [10]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data
# Data(edge_index=[2, 4], x=[3, 1])

Data(x=[3, 1], edge_index=[2, 4])

In [11]:

import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
data
# Data(edge_index=[2, 4], x=[3, 1])

Data(x=[3, 1], edge_index=[2, 4])

In [7]:
data.validate(raise_on_error=True)

True

In [9]:
print(data.keys())

['edge_index', 'x']


In [ ]:
print(data['x'])

In [12]:
for key, item in data:
    print(f'{key} found in data')

x found in data
edge_index found in data


In [13]:
'edge_attr' in data

False

In [14]:
data.num_nodes

3

In [15]:
data.num_edges

4

In [17]:
data.num_node_features

1

In [18]:
data.has_isolated_nodes()

False

In [19]:
data.has_self_loops()

False

In [20]:
data.is_directed()

False

In [16]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

In [31]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [32]:
dataset = dataset.shuffle()

In [33]:
len(dataset)

600

In [34]:
dataset.num_classes

6

In [35]:
dataset.num_node_features

3

In [36]:
data = dataset[0]

In [37]:
data.is_undirected()

True

In [38]:
train_dataset = dataset[:540]

In [39]:
test_dataset = dataset[540:]

In [40]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)
    print(batch.num_graphs)

DataBatch(edge_index=[2, 4374], x=[1171, 21], y=[32], batch=[1171], ptr=[33])
32
DataBatch(edge_index=[2, 4296], x=[1057, 21], y=[32], batch=[1057], ptr=[33])
32
DataBatch(edge_index=[2, 3864], x=[1007, 21], y=[32], batch=[1007], ptr=[33])
32
DataBatch(edge_index=[2, 4348], x=[1164, 21], y=[32], batch=[1164], ptr=[33])
32
DataBatch(edge_index=[2, 4106], x=[1076, 21], y=[32], batch=[1076], ptr=[33])
32
DataBatch(edge_index=[2, 4188], x=[1070, 21], y=[32], batch=[1070], ptr=[33])
32
DataBatch(edge_index=[2, 3734], x=[982, 21], y=[32], batch=[982], ptr=[33])
32
DataBatch(edge_index=[2, 3862], x=[1059, 21], y=[32], batch=[1059], ptr=[33])
32
DataBatch(edge_index=[2, 3878], x=[1015, 21], y=[32], batch=[1015], ptr=[33])
32
DataBatch(edge_index=[2, 3974], x=[1018, 21], y=[32], batch=[1018], ptr=[33])
32
DataBatch(edge_index=[2, 3784], x=[980, 21], y=[32], batch=[980], ptr=[33])
32
DataBatch(edge_index=[2, 3904], x=[1007, 21], y=[32], batch=[1007], ptr=[33])
32
DataBatch(edge_index=[2, 4432], 

In [41]:
from torch_geometric.utils import scatter
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    print(data)
    print(data.num_graphs)
    x = scatter(data.x, data.batch, dim=0, reduce='mean')
    print(x.size())

DataBatch(edge_index=[2, 4220], x=[1083, 21], y=[32], batch=[1083], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 3982], x=[1048, 21], y=[32], batch=[1048], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4074], x=[1071, 21], y=[32], batch=[1071], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4268], x=[1072, 21], y=[32], batch=[1072], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4050], x=[1050, 21], y=[32], batch=[1050], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4042], x=[1096, 21], y=[32], batch=[1096], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 3466], x=[929, 21], y=[32], batch=[929], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4176], x=[1108, 21], y=[32], batch=[1108], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4490], x=[1187, 21], y=[32], batch=[1187], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4208], x=[1071, 21], y=[32], batch=[1071], ptr=[33])
32
tor

In [45]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

dataset[0]

Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

In [50]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [51]:
dataset.num_node_features

1433

In [52]:
dataset.num_classes

7

In [53]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [55]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8050
